### Example: Using Enola for Dynamically Field-Programmable Qubit Arrays Based on Neutral Atoms

This notebook demonstrates how to use our compiler, Enola, to compile and execute quantum programs on a single-zone neutral atom architecture. The architecture is based on dynamically field-programmable qubit arrays, where qubits can be reconfigured during circuit execution.

***Prerequisite***
Download Enola from [Github](https://github.com/UCLA-VAST/Enola)

In [ ]:
!git clone https://github.com/UCLA-VAST/Enola.git
!pip3 install networkx
!pip3 install matplotlib
!pip3 install 'qiskit>=1'
!pip3 install ffmpeg-python

***Import***

In [24]:
from enola.enola import Enola
import qiskit

### Compile a Circuit

***Step 1: Compiler intialization***

- Default compiler: Use edge coloring algorithm for commutable gate scheduling, simulated annealing with dynamic placement, maximal independent set with sorting heuristic for routing strategy, no restriction for vectex number in solving MIS. We set `full_code=True` for animation code generation.   

- Scalable compiler: Use trivial layout (`trivial_layout=True`), return to initial mapping after each Rydberg stage (`reverse_to_initial=True`), using maximal independent set for routing (`routing_strategy='maximalis'`), and limit the number of vertices to 1000 in solving MIS (`use_window=True`)

In [20]:
commutable_circuit_name = 'qaoa'
default_compiler = Enola(
        name=commutable_circuit_name,
        dir='./results/',
        full_code=True)

scalable_compiler = Enola(
        name=commutable_circuit_name,
        dir='./results/',
        trivial_layout = True,
        routing_strategy="maximalis",
        reverse_to_initial=True,
        use_window=True,
        full_code=True)

***Step 2: Set Architecture***

Set architecture dimension by the method `setArchitecture(bounds: Sequence[int])`, where `bounds = [slm_num_row, slm_num_col, aod_num_row, aod_num_col]` specifies the number of columns and rows in the SLM and AOD arrays. Note that 1. the SLM site number, i.e., `slm_num_row`*`slm_num_col`, should be greater than or equal to the number of program qubits in the circuit, and 2. the size of AOD array should not be smaller than the size of SLM arrays.

In [ ]:
# an example of setting an architecture with an SLM of 10 columns and rows, and an AOD with 10 columns and rows
default_compiler.setArchitecture([10, 10, 10, 10])

***Step 3: Set Program***

Set quantum program by the method `setProgram(program: Sequence[Sequence[int]])`, where `program` is a list of two-qubit gates in the circuit.

In [ ]:
# an example of setting the QAOA circuit
# circuits with all commutable gates
commutable_circuit_name = "qaoa"
graph_edge_list = [[0, 1], [0, 2], [0, 4], [1, 3], [1, 5], [2, 4], [2, 3], 
                   [3, 5], [4, 5]]

default_compiler.setProgram(graph_edge_list)

***Step 4: Compile the Circuit***

Compile the circuit by method `solve()`. 
- `save_file` specifies whether to save 1. the output program list and 2. the detailed code (if `full_code=True` when initializing the compiler) for animation generation to json files in the folder `results/code`, and 3. the runtime analysis to a json file to the folder `results/time`.

In [ ]:
program_list_qaoa = default_compiler.solve(save_file=True)

#### The other example for compiling a generic circuit

Initialize a compiler for generic circuits with gate dependency by setting `dependency=True`.

In [ ]:
generic_circuit_name = "qasm_exp/ising_n26.qasm"
# generic circuits with dependencies
# extract two-qubit gates from the circuit
list_gate_two_qubit = []
with open(generic_circuit_name, 'r') as f:
    qasm_str = f.read()
    circuit = qiskit.QuantumCircuit.from_qasm_str(qasm_str)
    cz_circuit = qiskit.transpile(circuit, basis_gates=["cz", "id", "u2", "u1", "u3"])
    instruction = cz_circuit.data
    for ins in instruction:
        if ins.operation.num_qubits == 2:
            list_gate_two_qubit.append((ins.qubits[0]._index, ins.qubits[1]._index))
print("list_gate_two_qubit: ", list_gate_two_qubit)

# use the scalable compiler with dependency = True
scalable_generic_compiler = Enola(
        name="ising_n26",
        dir='./results/',
        trivial_layout = True,
        dependency=True,
        routing_strategy="maximalis",
        reverse_to_initial=True,
        use_window=True)

# set architecture
scalable_generic_compiler.setArchitecture([10, 10, 10, 10])

# set program
scalable_generic_compiler.setArchitecture(list_gate_two_qubit)

# solve the problem
program_list_generic = scalable_generic_compiler.solve(save_file=True)

### Circuit Fidelity Simulation
$f=(f_{1})^{g_1} \cdot \underbrace{(f_{2})^{g_2} \cdot (f_\text{exc})^{|Q| S - 2g_2}}_\text{two-qubit gate} \cdot \underbrace{(f_\text{trans})^{N_\text{trans}}}_\text{atom transfer} \cdot \underbrace{\Pi_{q\in Q}\ (1-T_q/T_2)}_\text{decoherence}$
- $f_{1}$: Single-qubit gate fidelity 
- $f_{2}$: Two-qubit gate fidelity 
- $f_\text{exc}$: Idle qubit excitation fidelity 
- $f_\text{trans}$: Atom transfer fidelity 
- $g_1$: Number of single-qubit gates
- $g_2$: Number of two-qubit gates
- $Q$: Set of program qubits
- $S$: Number of stage
- $N_\text{trans}$: Number of atom transfers
- $T_q$: Idle time for qubit $q$

***Run simulator***: 
Run `python3 simulator.py <P> --arch_param <A> --fidelity_param <F>` where `<P>` is the filename to the program instruction lists, `<A>` is the json file constaining architecture parameters, `<F>` is the json file containing fidelity parameters.

Default: 
- `fidelity_param` is "hardware_spec/compute_store_arch_fidelity.json"
    - R_B: 6um
    - AOD_SEP: 2um
    - RYD_SEP: 15um
    - SITE_SLMS: 2um
    - SLM_SEP: 2um
    - SITE_WIDTH: 4um
    - T_RYDBERG: 0.36us
    - T_ACTIVATE: 15us

- `arch_param` is "hardware_spec/compute_store_arch.json"
    - 2QG: 0.995,
    - 1QG: 0.9997,
    - AT: 0.999,
    - T: 1.5e6us

The program will save the json file consisting of fidelity anaylsis to the folder `results/fidelity/`

In [ ]:
# example command
!python3 simulator.py results/code/qaoa_code.json

### Animation Generation

To generate animation, run `python animation.py <F>` where `<F>` is the full code file. By default, we assume the input is a graph-induced circuit so there is a graph located next to the architecture to indicate the circuit execution condition. To remove the graph, add the argument `--noGraph`.

In [ ]:
# example command
!python3 animation.py results/code/qaoa_code_full.json
!python3 animation.py results/code/ising_n26_code_full.json --noGraph